In [ ]:
from bs4 import BeautifulSoup
from itertools import islice
import urllib
import csv
import re

# Open csv file and read rows into a dict
movie_list = {}
with open('movie_list.csv', 'r') as csvfile:
    reader = csv.DictReader(csvfile)
    for rows in reader:
        # Remove punctuation except "'s" e.g. Winter's Bone
        rows['movie_title'] = re.sub(r'[,:;!?/-]', '', rows['movie_title'])
        movie_list[rows['movie_title']] = rows['release_year']
        
'''def take(n, iterable):
    "Return first n items of the iterable as a list"
    return list(islice(iterable, n))
 
n_items = take(3, movie_list.iteritems())
 
print n_items'''

print movie_list

In [ ]:
def get_imdbid(movie_list):
    '''
    Retrieve IMDb IDs from movie list via OMDb API
    '''
    imdbid = {}
    for title, year in movie_list.iteritems():
        # Add + in between spaces in movie title
        title_query = re.sub(r'\s', '+', title)
        # Extract IMDb IDs
        r = urllib.urlopen('http://www.omdbapi.com/?t=' 
                           + title_query + '&y=' + year + '&plot=short&r=xml').read()
        soup = BeautifulSoup(r, 'xml')
        # Handle invalid movie title/year search
        if soup.error:
            print title + " not found"
        else:
            imdbid[title] = soup.movie['imdbID']
    return imdbid

imdbid = get_imdbid(movie_list)
print imdbid

In [ ]:
def get_awards(imdbid, award_list):
    '''
    Scrape list of awards nominated and won for each movie title
    '''
    dataset = {}
    for title, id in imdbid.iteritems():
        # Add movie titles in dataset
        dataset[title] = {}
        r = urllib.urlopen('http://www.imdb.com/title/' + id + '/awards?ref_=tt_awd').read()
        soup = BeautifulSoup(r, 'lxml')
        awards = soup.find("div", class_="article listo")
        for a in awards.contents:
            for award in award_list:
                # Search for h3 tags which contain the award title
                # NavigableString element is converted to unicode string to save memory
                if a.name == "h3" and unicode(a.next_element).strip() == award:
                    # Search award results in its next sibling
                    result = a.find_next_sibling().td.b
                    # If award is won, add award = 1 to dataset
                    if result.get_text() == "Won":
                        dataset[title][award] = 1
                else:
                    # Add award = 0 if not won
                    try:
                        dataset[title][award] != 1
                    except:
                        dataset[title][award] = 0
    return dataset
    
award_list = ["Screen Actors Guild Awards", "Directors Guild", "Producers Guild"]       
dataset = get_awards(imdbid, award_list)
#print dataset